# Tutorial: Building a Recency-Aware Question-Answering System with Llama-Index


In this tutorial, we will learn how to build a question-answering system that is aware of the recency of information. This is especially important for questions whose answers may change over time, such as "Who is the current US President?".

We will be using the llama_index package, a powerful tool for building large-scale information retrieval systems.

## Installing necessary packages
Before we start, make sure that you have the llama_index package installed. You can install it using pip:

In [1]:
# !pip install llama-index 

You will also need the qdrant_client package for this tutorial. You can install it via pip as well:

In [2]:
# !pip install qdrant-client

In [3]:
# !pip install 'rich[jupyter]'
# from rich import print
%load_ext rich

In [4]:
import qdrant_client
from datetime import datetime
from llama_index import GPTVectorStoreIndex

from llama_index.vector_stores.qdrant import QdrantVectorStore
from pathlib import Path
from llama_index import GPTVectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.indices.postprocessor import (
    FixedRecencyPostprocessor,
    EmbeddingRecencyPostprocessor,
)
# load documents
from llama_index.storage.storage_context import StorageContext
Path.ls = lambda x: list(x.iterdir())

# Loading Documents

First, let's load our documents. In this example, we will use the News Category Dataset v3. This dataset contains news articles with various fields like `headline`, `category`, `short_description`, `link`, `authors`, and date. We'll load the data, and reformat it to suit our needs.

In [5]:
import json

with open("../data/News_Category_Dataset_v3.json", "r") as f:
    data = [json.loads(k) for k in f.readlines()]
    links = [k.pop("link") for k in data]
    authors = [k.pop("authors") for k in data]
    data = [
        {
            "text": f"{k['headline']} under the category: {k['category']}\n {k['short_description']}",
            "date": k["date"],
        }
        for k in data
    ]

In [6]:
import datetime
dates = [datetime.datetime.fromisoformat(element["date"]) for element in data]
min(dates), max(dates)

(datetime.datetime(2012, 1, 28, 0, 0), datetime.datetime(2022, 9, 23, 0, 0))

Next, we'll write these documents to text files in a directory. Each document will be written to a text file named by its date.

In [7]:
from pathlib import Path

write_dir = Path("../data/dump").resolve()
write_dir.mkdir(exist_ok=True, parents=True)
for element in data:
    file_path = write_dir / f"{element['date']}.txt"
    with file_path.open("w") as f:
        f.write(element["text"])


This is what we want to build:

This diagram shows that:

1. The user issues a query to the query engine.
2. The query engine, which has been configured with certain postprocessors, performs a search on the vector store based on the query.
3. The query engine then postprocesses the results.
4. The postprocessed results are then returned to the user.

Each arrow represents the direction of data flow. The "Query Engine" box encapsulates the postprocessing step to indicate that it's a part of the query engine's function. This diagram is meant to provide a high-level understanding of the process and does not include all the details involved.


We're using Llama Index to make the Query Engine and Qdrant for our Vector Store



```
+-----------+          +------------------+          +-----------------+
|           |  Query   |                  |  Search  |                 |
|  User     +--------->+  Query Engine    +--------->+     Qdrant      |
|           |          |                  |          |   Vector Store  |
|           |          | (with            |          |                 |
+-----------+          | Postprocessors)  |          +-----------------+
                       |                  |          |                 |
                       |  +------------+  |          |                 |
                       |  |Postprocess |  |  Top K   |                 | 
                       |  |            |  |Candidates|                 |
                       +--|Candidates  |  +<---------+                 |
                       |  +------------+  |                            |
                       |                  |                            |
+-----------+          +------------------+                            |
|           |   Final  |                                               |
|  User     |<---------+                                               |
|           |  Answer  |                                               |
+-----------+                                                          |
                                                                       |
+---------------------------------------------------------------------+
```

## Creating Qdrant Client
We'll be using Qdrant as our vector storage system. Qdrant is a high-performance vector database designed for storing and searching large-scale high-dimensional vectors.

- You can use `:memory:` mode for fast and lightweight experiments. It does not require Qdrant to be deployed anywhere but requires `qdrant-client >= 1.1.1`.

- Otherwise, set the Qdrant instance address with:
  - `uri="http://<host>:<port>"`

- Set the API KEY for Qdrant Cloud:
  - `api_key="<qdrant-api-key>"`

In [8]:
client = qdrant_client.QdrantClient(
    location="/tmp/huffpost_news"
)

## Loading Data into Llama-Index
Llama-Index has a simple way to load documents from a directory. We can define a function to get the metadata from a file name, and pass this function to the `SimpleDirectoryReader` class.

In [9]:
def get_file_metadata(file_name: str):
    """Get file metadata."""
    return {"date": Path(file_name).stem}


documents = SimpleDirectoryReader(
    input_files=write_dir.ls(), file_metadata=get_file_metadata
).load_data()

We then parse these documents into nodes and create our QdrantVectorStore

In [10]:
from llama_index import GPTVectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.vector_stores.qdrant import QdrantVectorStore

# define service context (wrapper container around current classes)
service_context = ServiceContext.from_defaults(chunk_size_limit=512)

# use node parser in service context to parse into nodes
nodes = service_context.node_parser.get_nodes_from_documents(documents)
vector_store = QdrantVectorStore(client=client, collection_name="huffpostnews")

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /opt/homebrew/Caskroom/miniconda/base/envs/qdrant/lib/python3.11/site-packages/httpcore/_excepti │
│ ons.py:10 in map_exceptions                                                                      │
│                                                                                                  │
│    7 @contextlib.contextmanager                                                                  │
│    8 def map_exceptions(map: ExceptionMapping) -> Iterator[None]:                                │
│    9 │   try:                                                                                    │
│ ❱ 10 │   │   yield                                                                               │
│   11 │   except Exception as exc:  # noqa: PIE786                                                │
│   12 │   │   for from_exc, to_exc in map.items():                                                │
│   13 │   │   │   if isinstance(exc, from_exc):                                                   │
│                                                                                                  │
│ /opt/homebrew/Caskroom/miniconda/base/envs/qdrant/lib/python3.11/site-packages/httpcore/backends │
│ /sync.py:94 in connect_tcp                                                                       │
│                                                                                                  │
│    91 │   │   │   OSError: ConnectError,                                                         │
│    92 │   │   }                                                                                  │
│    93 │   │   with map_exceptions(exc_map):                                                      │
│ ❱  94 │   │   │   sock = socket.create_connection(                                               │
│    95 │   │   │   │   address, timeout, source_address=source_address                            │
│    96 │   │   │   )                                                                              │
│    97 │   │   return SyncStream(sock)                                                            │
│                                                                                                  │
│ /opt/homebrew/Caskroom/miniconda/base/envs/qdrant/lib/python3.11/socket.py:827 in                │
│ create_connection                                                                                │
│                                                                                                  │
│   824 │                                                                                          │
│   825 │   host, port = address                                                                   │
│   826 │   exceptions = []                                                                        │
│ ❱ 827 │   for res in getaddrinfo(host, port, 0, SOCK_STREAM):                                    │
│   828 │   │   af, socktype, proto, canonname, sa = res                                           │
│   829 │   │   sock = None                                                                        │
│   830 │   │   try:                                                                               │
│                                                                                                  │
│ /opt/homebrew/Caskroom/miniconda/base/envs/qdrant/lib/python3.11/socket.py:962 in getaddrinfo    │
│                                                                                                  │
│   959 │   # We override this function since we want to translate the numeric family              │
│   960 │   # and socket type values to enum constants.                                            │
│   961 │   addrlist = []                                                                          │
│ ❱ 962 │   for res in _socket.getaddrinfo(host, port, family

Next, we'll create our `GPTVectorStoreIndex` from the documents. This operation might take some time as it's creating the index from the documents.

In [ ]:
%%time
from llama_index.storage.storage_context import StorageContext

storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = GPTVectorStoreIndex.from_documents(documents, storage_context=storage_context)

## Defining Recency Postprocessors

Llama-Index allows us to add postprocessors to our query engine. These postprocessors can modify the results of our queries after they are returned from the index. Here, we'll add a recency postprocessor to our query engine. This postprocessor will prioritize recent documents in the results.

We'll define two types of recency postprocessors - `FixedRecencyPostprocessor` and `EmbeddingRecencyPostprocessor`.

In [ ]:
recency_postprocessor = FixedRecencyPostprocessor(service_context=service_context)

# Incorporating Reranking using Cohere
Cohere is a powerful AI model that can help us rerank our results. We'll define a `CohereRerank` postprocessor and add it to our query engine.

First, install the cohere package:

In [ ]:
# !pip install cohere

Using it in our code:

In [ ]:
import os
from llama_index.indices.postprocessor.cohere_rerank import CohereRerank

## Defining Query Engines
We'll define four query engines for this tutorial: 
1. Just the Vector Store i.e. Qdrant here
1. A recency query engine
1. A reranking query engine
1. And a combined query engine.

The recency query engine uses the `FixedRecencyPostprocessor`, the reranking query engine uses the `CohereRerank` postprocessor, and the combined query engine uses both.

In [ ]:
from pydantic import BaseModel
from typing import Optional

class Interaction(BaseModel):
    question: str
    answer: Optional[str] = None
    response: Optional[object] = None

In [ ]:
top_k = 10

In [ ]:
index_query_engine = index.as_query_engine(
    similarity_top_k=top_k,
) 

In [ ]:
recency_query_engine = index.as_query_engine(
    similarity_top_k=top_k,
    node_postprocessors=[recency_postprocessor],
) 

In [ ]:
cohere_rerank = CohereRerank(api_key=os.environ["COHERE_API_KEY"], top_n=top_k)
reranking_query_engine = index.as_query_engine(
    similarity_top_k=top_k,
    node_postprocessors=[cohere_rerank],
) 

In [ ]:
query_engine = index.as_query_engine(
    similarity_top_k=top_k,
    node_postprocessors=[cohere_rerank, recency_postprocessor],
)

## Querying the Engine
Finally, we can query our engine. Let's ask it "Who is the current US President?" and see the results from each query engine.

In [ ]:
question = "Who is the current US President?"

def compare_responses(question: QA)->None:
    responses = {}
    print("*"*len(question))
    print(f"Question: {question}")
    print("*"*len(question))
    response = index_query_engine.query(question)
    responses["qdrant"] = response
    print("No Postprocessor, Qdrant:", response)
    response = recency_query_engine.query(question)
    responses["recency"] = response
    print("Recency:", response)
    response = reranking_query_engine.query(question)
    responses["reranking"] = response
    print("Reranking:", response)
    response = query_engine.query(question)
    responses["both"] = response
    print("Both:", response)
    return responses

responses = compare_responses(question)
"Correct answer is Joe Biden"

In [ ]:
question = "When did Mughals invade India?"
# responses = compare_responses(question)
# 

print("**Correct answer** is Babur's victory in Battle of Panipat in April 1526")
# responses["both"].source_nodes, responses["qdrant"].source_nodes

In [ ]:
question = "Who won the latest Superbowl?"
compare_responses(question)
"Correct answer is Kansas Chiefs in May 2023, but the data cutoff is September 2022 for which Los Angeles Rams is the correct answer"

In [ ]:
question = "When did Kargil war happen? Why is that significant?"
compare_responses(question)
"1999, 2 nuclear countries went to war for the first time"

In [ ]:
question = "Who is the Indian Prime Minister in January 2014?"
responses = compare_responses(question)